In [2]:
%reload_ext autoreload
%autoreload 2

from model import GPT, GPTConfig
import torch
from utils import *

from mingpt_utils import set_seed
from mingpt_utils import sample_new, sample
import pytz
from datetime import datetime, timezone
import numpy as np
torch.cuda.empty_cache()

In [3]:
tokens = np.load('../data/formatted/tokens.npy', allow_pickle=True)
train = np.load('../data/shuffled/dataset_train.npy', allow_pickle=True)
midi_train = np.load('../data/shuffled/midi_train.npy', allow_pickle=True)
print(train.shape, midi_train.shape)

(43272, 1024) (43272, 1024, 8)


In [7]:

for e in train[0]:
    print(e, end=' ')

<style> Samba Tonality B minor <start> |: . 4.0 B m | . 4.0 F# dom7 | . 4.0 B m | . 4.0 B dom7 | . 4.0 B dom7 | . 4.0 E m | . 4.0 A dom7 | . 4.0 D maj7 | . 2.0 C# m7 alter b5 . 2.0 F# dom7 | . 4.0 B m | . 4.0 F# dom7 | . 4.0 B m | . 4.0 B dom7 | . 4.0 E m | . 4.0 E m | . 4.0 B m | . 2.0 G dom7 . 2.0 F# dom7 | . 2.0 B m . 2.0 D# o b|| . 4.0 E m | . 4.0 B m | . 2.0 G dom7 . 2.0 F# dom7 | . 2.0 B m . 2.0 B dom7 :| |: . 4.0 B m | . 4.0 F# dom7 | . 4.0 B m | . 4.0 B dom7 | . 4.0 B dom7 | . 4.0 E m | . 4.0 A dom7 | . 4.0 D maj7 | . 2.0 C# m7 alter b5 . 2.0 F# dom7 | . 4.0 B m | . 4.0 F# dom7 | . 4.0 B m | . 4.0 B dom7 | . 4.0 E m | . 4.0 E m | . 4.0 B m | . 2.0 G dom7 . 2.0 F# dom7 | . 2.0 B m . 2.0 D# o b|| . 4.0 E m | . 4.0 B m | . 2.0 G dom7 . 2.0 F# dom7 | . 2.0 B m . 2.0 B dom7 :| |: . 4.0 B m | . 4.0 F# dom7 | . 4.0 B m | . 4.0 B dom7 | . 4.0 B dom7 | . 4.0 E m | . 4.0 A dom7 | . 4.0 D maj7 | . 2.0 C# m7 alter b5 . 2.0 F# dom7 | . 4.0 B m | . 4.0 F# dom7 | . 4.0 B m | . 4.0 B dom7 | . 

In [8]:
block_size = 1024
dataset = TokenDatasetMidi(train, midi_train, block_size, tokens)

data has 43272 pieces, 198 unique tokens.


In [9]:
epochs = 120
embedding = 256
heads = 4
layers = 4
batch_size = 128
learning_rate = 3e-5
num_workers = 4
midi_vocab = 128
token_size = len(tokens)

mconf = GPTConfig(token_size, block_size, midi_vocab, n_layer=layers, n_head=heads, n_embd=embedding)
session_model = GPT(mconf)

MODEL_NAME = "../models/model_"+ "epochs->" + str(epochs) + "_heads->" + str(heads) + "_embd->" + str(embedding) + "_batch->" + str(batch_size) + "_new_midi_embeddings"

session_model = load_model(MODEL_NAME, session_model)

03/18/2024 10:17:44 - INFO - model -   number of parameters: 3.269632e+06


Checkpoint loaded ../models/model_epochs->120_heads->4_embd->256_batch->128_new_midi_embeddings


In [10]:
import formats as fmt
import voicing as vc
voicing = vc.Voicing()

def generateSample(context, duration, style, tonality, session_model, dataset, temperature=1.0, sample=True, top_k=None, top_p=0.99):
    data, _ = fmt.getArrayOfElementsInChord(context, duration)
    data = ['<style>'] + [style] + ['Tonality'] + [tonality] + ['<start>'] + ['|'] + data
    
    midi, _ = voicing.get_midi(data)
    # for d, m in zip(data, midi):
    #     print(d, m)

    i = 0
    while ( i < 200):    
        x = torch.tensor([dataset.stoi[s] for s in data], dtype=torch.long)[None,...].to('cuda')
        m = torch.tensor(midi, dtype=torch.long)[None,...].to('cuda')
        
        #print(x.shape, m.shape)
        y = sample_new(session_model, x, m, 1, temperature=temperature, sample=sample, top_k=top_k, top_p=top_p)[0]
        
        data = [dataset.itos[int(i)] for i in y if dataset.itos[int(i)]]
        
        if len(data) > 2:
            if data[-1] == data[-2]:
                print("Duplicated element: ", data[-1], data[-2])
                data = data[:-1]
                
        if data[-2] == '.' and data[-1] not in voicing.durations:
            print("Durations are not correct: ", data[-1], data[-2])
            data = data[:-2]
            
        if data[-2] in voicing.durations and data[-1] not in voicing.all_notes:
            print("Note is not correct: ", data[-1], data[-2])
            data = data[:-2]
            
        #print(data)
        midi, status = voicing.get_midi(data)
        if status == False:
            #erase the last element
            print("Error creating the MIDI format")
            break
        i+=1 

    #myChords = convertChordsFromOutput(data)
    print(data)
    return data

In [23]:
context = ['C']
duration = np.full(len(context), 4.0, dtype=float)
myStyle = 'Jazz'
tonality = 'C major'
data = generateSample(context, duration, myStyle, tonality, session_model, dataset, temperature=1.0, sample=True, top_k=None, top_p=0.98)

['<style>', 'Jazz', 'Tonality', 'C major', '<start>', '|', '.', '4.0', 'C', 'maj6', 'add 9', '|', '.', '4.0', 'C', 'maj6', 'add 9', '|', '.', '4.0', 'A', 'm7', 'alter b5', '|', '.', '4.0', 'D', 'dom7', 'add #11', '|', '.', '4.0', 'D', 'm7', '|', '.', '4.0', 'E', 'm7', 'alter b5', '|', '.', '4.0', 'A', 'dom7', 'add b13', '|', '.', '4.0', 'D', 'm7', '|', '.', '2.0', 'G', 'dom7', '.', '2.0', 'E', 'dom7', '|', '.', '4.0', 'A', 'dom7', 'add b13', '|', '.', '4.0', 'D', 'm7', '|', '.', '4.0', 'C', 'maj6', 'add 9', '|', '.', '4.0', 'F#', 'm7', '|', '.', '4.0', 'C', 'maj6', 'add 9', '|', '.', '4.0', 'A', 'dom7', 'add b13', '|', '.', '4.0', 'D', 'm7', '|', '.', '4.0', 'B', 'm7', 'alter b5', '|', '.', '4.0', 'E', 'dom7', 'add b13', '|', '.', '4.0', 'A', 'm7', '|', '.', '2.0', 'A', 'm7', 'alter b5', 'add b9', '.', '2.0', 'D', 'dom7', 'alter b5', 'add b9', '|', '.', '4.0', 'G', 'm7', '|', '.', '2.0', 'G', 'm7', '.', '2.0', 'C', 'dom7', '|', '.', '4.0', 'F', 'maj6', 'add 9', '|', '.', '4.0', 'C', 'm

In [24]:
midi, _ = voicing.convert_chords_to_voicing(data)
voicing.export_to_midi(midi, "generated_"+myStyle)

song: generated_Jazz
MIDI file created! 
---------------------------------
